# Sub-Numpy
We will create our own implementation of a few functionalities supported by the NumPy
Library. We will call our implementation SNumPy (for Sub-NumPy). SNumPy will be the name of the
class you implement, and we will refer to it by the shorthand “snp” from here on

In [49]:

class Validator:
    """
    Validator Class that validates the input data and ensures good error messages.
    """

    @staticmethod
    def _shape(array):
        """
        Helper method that returns the shape of the input array.

        Args:
            array: The array to be checked.

        Returns:
            tuple: The shape of the array.
        """
        # If it's a vector, return the length of the vector
        if not array or not isinstance(array[0], list):
            return (len(array),)
        # Else, it's a matrix (2D array)
        else:
            return (len(array), len(array[0]))
        

    @staticmethod
    def is_vector(array):
        """
        Checks if the input is a vector.

        Args:
            array: The array to be checked.

        Returns:
            bool: True if the input is a vector, False otherwise.
        """
        # Check if it's a list type
        if not isinstance(array, list):
            raise TypeError(
                f"Input must be a list, but received {type(array).__name__}.")
        # Check if it's a single list
        if not len(Validator._shape(array)) == 1:
            return False
        # Check if all elements are numeric
        if not all(isinstance(item, (int, float)) for item in array):
            raise ValueError(
                "All elements must be numeric (int or float).")
        
        return True

    @staticmethod
    def is_matrix(array):
        """
        Checks if the input is a matrix (i.e., a list of lists).

        Args:
            array: The array to be checked.

        Returns:
            bool: True if the input is a matrix, False otherwise.
        """
        # Check if it's a list type
        if not isinstance(array, list):
            raise TypeError(
                f"Input must be a list, but received {type(array).__name__}.")
        # Check if it's a list of lists
        if not len(Validator._shape(array)) >= 2:
            return False
        # Check if all elements are numeric
        if not all(isinstance(item, (int, float)) for row in array for item in row):
            raise ValueError(
                "All elements must be numeric (int or float).")
        # Check if all rows have the same length
        if not all(len(row) == len(array[0]) for row in array):
            raise ValueError(
                "All rows must have the same length.")

        return True

    @staticmethod
    def is_vector_or_matrix(array):
        """
        Validates whether the input is a vector or a matrix.

        Args:
            array (list): The array to be checked.

        Returns:
            bool: True if the input is a vector or matrix, False otherwise.

        Raises:
            ValueError: If the input is not a list or a mixed-type list.
        """
        # Use the other validators to check if it's a vector or matrix
        if Validator.is_vector(array) or Validator.is_matrix(array):
            return True
        else:
            raise ValueError(
                f"Input must be either a vector or a matrix, but received shape {Validator._shape(array)}.")

    @staticmethod
    def validate_shape_for_operations(array1, array2, operation_type):
        """
        Validates the shape of arrays for various operations.

        Args:
            array1 (list): The first array.
            array2 (list): The second array.
            operation_type (str): The type of operation (e.g., 'add', 'multiply').

        Raises:
            ValueError: If the shapes are not compatible for the operation.
        """
        shape1 = Validator._shape(array1)
        shape2 = Validator._shape(array2)

        #Check if input is a vector or matrix
        Validator.is_vector_or_matrix(array1)
        Validator.is_vector_or_matrix(array2)

        if operation_type == 'append':
            # Check if both arrays are vectors or matrices
            if len(shape1) != len(shape2):
                raise ValueError(
                    f"Cannot append a vector to a matrix or vice versa. Got shapes: {shape1} and {shape2}.")

            # If both are matrices, check if they have the same number of rows
            if len(shape1) == 2 and shape1[0] != shape2[0]:
                raise ValueError(
                    f"Matrices must have the same number of rows to append. Got rows: {shape1[0]} and {shape2[0]}.")

        # Check if the shapes are compatibale for addition and subtraction
        if operation_type in ['add', 'subtract'] and shape1 != shape2:
            raise ValueError(
                f"Shapes must be the same for addition and subtraction. Got shapes: {shape1} and {shape2}.")

        # Check if the shapes are compatible for dot product
        if operation_type == 'dotproduct':
            # If both are vectors, check if they have the same length
            if Validator.is_vector(array1) and Validator.is_vector(array2):
                if shape1 != shape2:
                    raise ValueError(
                        f"Vectors must be the same size for dot product. Vector1 length: {shape1[0]}, Vector2 length: {shape2[0]}.")
            # If both are matrices, check if Array1 have the same number of columns as Array2 rows
            elif Validator.is_matrix(array1) and Validator.is_matrix(array2):
                if shape1[1] != shape2[0]:
                    raise ValueError(
                        f"The number of columns in Matrix1 (shape: {shape1}) must match the number of rows in Matrix2 (shape: {shape2}) for dot product.")
            # If Array1 is a vector and Array 2 is a matrix, check if the vector length matches the matrix rows
            elif (Validator.is_vector(array1) and Validator.is_matrix(array2)) or (Validator.is_matrix(array1) and Validator.is_vector(array2)):
                if len(shape1) == 1:
                    if shape1[0] != shape2[0]:
                        raise ValueError(
                            f"Vector length (length: {shape1[0]}) must match the number of rows in the matrix (shape: {shape2}) for dot product.")
                # If Array1 is a matrix and Array2 is a vector, check if the matrix columns match the vector length
                else:
                    if shape1[1] != shape2[0]:
                        raise ValueError(
                            f"The number of columns in the matrix (shape: {shape1}) must match the vector length (length: {shape2[0]}) for dot product.")

        # Check if the shapes are compatible for gaussian elimination
        if operation_type == 'gaussian_elimination':
            # Check if Array1 is a matrix
            if not Validator.is_matrix(array1):
                raise ValueError(
                    f"Matrix must be a matrix for gaussian elimination. Got shape: {shape1}.")
            # Check if the matrix is a square matrix
            if shape1[0] != shape1[1]:
                raise ValueError(
                    f"Matrix must be a square matrix for gaussian elimination. Got shape: {shape1}.")
            # Check if Array2 is a vector
            if not Validator.is_vector(array2):
                raise ValueError(
                    f"Vector must be a vector for gaussian elimination. Got shape: {shape2}.")
            # Check if the matrix rows are the as vector length
            if shape1[0] != shape2[0]:
                raise ValueError(
                    f"Matrix and vector must have the same number of rows for gaussian elimination. Got shapes: {shape1} and {shape2}.")

    @staticmethod
    def validate_index_for_get(array, index):
        """
        Validates the index for accessing elements in an array.

        Args:
            array (list): The array.
            index (tuple): The index tuple (row, column).

        Raises:
            ValueError: If the index is not a tuple of correct length.
            IndexError: If the index is out of bounds.
        """
        # If it's a vector, check if the index is an integer
        if Validator.is_vector(array):
            column = index[0]

            if not isinstance(index, (tuple, int)) or len(index) != 1:
                raise ValueError("Index must be a tuple of length 1.")
            
            if column >= len(array):
                raise IndexError(
                    f"Index {index} is out of bounds for array of shape {Validator._shape(array)}.")
            return
        
        # Unpack the index tuple
        row, column = index

        # Check if the index is a tuple of length 2 
        if not isinstance(index, (tuple, int)) or len(index) != 2:
            raise ValueError("Index must be a tuple of length 2.")

        # Check if the index is within the bounds of the array
        if row >= len(array) or (isinstance(array[0], list) and column >= len(array[0])):
            raise IndexError(
                f"Index {index} is out of bounds for array of shape {Validator._shape(array)}.")
        
        return
        

    @staticmethod
    def validate_shape_for_reshape(array, new_shape):
        """
        Validates if a vector can be reshaped to the new shape.

        Args:
            array (list): The vector to be reshaped.
            new_shape (tuple): The new shape as a tuple (rows, columns).

        Raises:
            ValueError: If reshaping is not feasible due to incompatible sizes or if input is not a vector.
        """
        # Check if the input is a vector
        if not Validator.is_vector(array):
            raise ValueError(
                f"Reshape operation is only valid for vectors, but received shape {Validator._shape(array)}.")
        
        # Check if the new shape is a tuple of length 2
        if len(new_shape) != 2:
            raise ValueError(
                f"New shape must be a tuple of length 2, but received shape {new_shape}.")
        # Check if the new shape is a tuple of positive integers
        if not isinstance(new_shape[0], int) or not isinstance(new_shape[1], int) or new_shape[0] <= 0 or new_shape[1] <= 0:
            raise ValueError(
                f"New shape must be a tuple of positive integers, but received shape {new_shape}.")
        
        # Check if the new shap can fit all elements of the vector
        total_elements = len(array)
        if total_elements != new_shape[0] * new_shape[1]:
            raise ValueError(
                f"Total elements mismatch for the new shape. Expected {new_shape[0] * new_shape[1]} elements, but got {total_elements}.")

    @staticmethod
    def validate_positive_integer(value):
        """
        Validates that the provided value is a positive integer.

        Args:
            value: The value to be validated.

        Raises:
            ValueError: If the value is not a positive integer.
        """
        # Check if the value is a positive integer
        if not isinstance(value, int) or value <= 0:
            raise ValueError(f"The value {value} is not a positive integer.")

In [43]:

class SNumPy:
    """
    SNumPy class for basic array manipulations.
    """

    @staticmethod
    def _create_array(n, m, value):
        """
        Create an array of given shape (n,m) with the given value.
        Helper method for ones() and zeros().

        Args:
            n: The number of rows.
            m: The number of columns.
            value: The value to be filled.

        Returns:
            A list of lists with the given value.
        """
        # Validate the input
        Validator.validate_positive_integer(n)
        if m is not None:
            Validator.validate_positive_integer(m)

        # Create the array with list comprehension
        if m is None:
            return [value for _ in range(n)]
        else:
            # If m is not none, then create a 2D array
            return [
                [value for _ in range(m)] 
                for _ in range(n)
            ]

    @staticmethod
    def ones(n: int, m: int = None) -> list:
        """
        Return an array of ones in given shape (n,m).

        Args:
            n: The number of rows. If m is None, then n is the number of columns.
            m: The number of columns (default is None).

        Returns:
            A list of ones in the given shape.
        """
        return SNumPy._create_array(n, m, 1)

    @staticmethod
    def zeros(n: int, m: int = None) -> list:
        """
        Return an array of zeros in given shape (n,m).

        Args:
            n: The number of rows. If m is None, then n is the number of columns.
            m: The number of columns (default is None).

        Returns:
            A list of zeros in the given shape.
        """
        return SNumPy._create_array(n, m, 0)

    @staticmethod
    def reshape(array: list, shape: tuple) -> list:
        """
        Return an array containing the same data with a new shape.
        Only vectors can be reshaped.

        Args:
            array: The array to be reshaped (vector).
            shape: The new shape as a tuple (rows, columns).

        Returns:
            Array in the new shape.
        """
        # Validate the input
        Validator.validate_shape_for_reshape(array, shape)

        # Unpack the shape tuple
        row, column = shape

        # Slice the vector into the new shape for each row in 'shape'
        return [array[i * column: (i + 1) * column] for i in range(row)]

    @staticmethod
    def shape(array: list) -> tuple:
        """
        Return the shape of an array.

        Args:
            array: The array to be checked.

        Returns:
            A tuple containing the shape of the array.
        """
        # Validate the input
        Validator.is_vector_or_matrix(array)

        # Check if it's a vector (1D array)
        if Validator.is_vector(array):
            return (len(array),)
        # Else, it's a matrix (2D array)
        else:
            return (len(array), len(array[0]))

    @staticmethod
    def append(array1: list, array2: list, axis: int = 0) -> list:
        """
        Returns a new array that is the combination of the two input arrays.

        Args:
            array1: The first vector/matrix.
            array2: The second vector/matrix.
            axis: The axis along which to append. 0 for row and 1 for column (default is 0).

        Returns:
            A new array appended along the given axis.
        """
        # Validate the input
        Validator.validate_shape_for_operations(array1, array2, 'append')

        if axis == 0:
            return array1 + array2  # Append by Row
        else:
            # Append by Column
            return [array1[i] + array2[i] for i in range(len(array1))]

    @staticmethod
    def get(array: list, index: tuple) -> int:
        """
        Return the element at the given index.

        Args:
            array: The array to be accessed.
            index: The index tuple (row, column).

        Returns:
            The element at the given index.
        """
        # Validate the input
        Validator.is_vector_or_matrix(array)
        Validator.validate_index_for_get(array, index)

        # If it's a vector, return the element at the index
        if Validator.is_vector(array):
            return array[index[0]]
        # Else, it's a matrix
        row, column = index  # Unpack the index tuple
        return array[row][column]

    @staticmethod
    def _elementwise_operation(array1, array2, operation):
        """
        Helper method to perform element-wise operations on two arrays.

        Args:
            array1 (list): The first array.
            array2 (list): The second array.
            operation (function): A lambda function that specifies the operation (addition or subtraction).

        Returns:
            list: The result of the element-wise operation.
        """
        # Validate the input shapes
        Validator.validate_shape_for_operations(
            array1, array2, operation.__name__)

        # Vector operation
        if Validator.is_vector(array1) and Validator.is_vector(array2):
            # The zip() function retuns a tuple of the corresponding elements in the two arrays
            return [operation(element1, element2) for element1, element2 in zip(array1, array2)]
    
        # Else, Matrix operation
        else:
            # First find each row and then each element in the row and perform the operation on them
            return [
                [operation(element1, element2)
                 for element1, element2 in zip(row1, row2)]
                for row1, row2 in zip(array1, array2)
            ]

    @staticmethod
    def add(array1: list, array2: list) -> list:
        """
        Return the element-wise sum of two arrays.

        Args:
            array1: The first array (vector or matrix).
            array2: The second array.

        Returns:
            NewArray = Array1 + Array2
        """
        return SNumPy._elementwise_operation(array1, array2, lambda x, y: x + y)  # Using lambda function to specify the operation (addition)

    @staticmethod
    def subtract(array1: list, array2: list) -> list():
        """
        Return the element-wise difference of two arrays.

        Args:
            array1: The first array (vector or matrix).
            array2: The second array (vector or matrix).

        Returns:
            NewArray = Array1 - Array2
        """
        return SNumPy._elementwise_operation(array1, array2, lambda x, y: x - y)  # Using lambda function to specify the operation (subtraction)

    @staticmethod
    def dotproduct(array1: list, array2: list) -> list:
        """
        Perform the dot product operation between two vectors, two matrices, or a vector and a matrix.

        Args:
            array1: The first vector/matrix.
            array2: The second vector/matrix.

        Returns:
            The dot product as a vector or matrix.

        Raises:
            ValueError: If the dimensions are not compatible for dot product.
        """
        # Validate the input
        Validator.validate_shape_for_operations(array1, array2, 'dotproduct')

        # Get shapes for operations
        shape1 = SNumPy.shape(array1)
        shape2 = SNumPy.shape(array2)

        # Vector multiplication
        if Validator.is_vector(array1) and Validator.is_vector(array2):
            return sum(array1[i] * array2[i] for i in range(len(array1)))

        # Vector | Matrix multiplication
        elif Validator.is_vector(array1) and Validator.is_matrix(array2):
            # Vector (treated as row vector) and Matrix results in a row vector
            return [
                sum(array1[k] * array2[k][j]
                    for k in range(len(array1)))
                for j in range(shape2[1])
            ]

        # Matrix | Vector multiplication
        elif Validator.is_matrix(array1) and Validator.is_vector(array2):
            # Matrix and Vector (treated as column vector) results in a column vector
            return [
                sum(array1[i][k] * array2[k] 
                    for k in range(shape1[1])) 
                for i in range(shape1[0])
            ]

        # Matrix multiplication
        elif Validator.is_matrix(array1) and Validator.is_matrix(array2):
            return [
                # For each row in array1, multiply each element in the row with the corresponding element in the column of array2
                # Results in array of (array1 rows * array2 columns)
                [sum(array1[i][k] * array2[k][j]
                     for k in range(shape1[1]))
                 for j in range(shape2[1])]
                for i in range(shape1[0])
            ]
        else:
            raise ValueError("Invalid input types for dot product")

    @staticmethod
    def scalar_multiply(array: list, scalar: int) -> list:
        """
        Return the scalar product of an array.

        Args:
            array: The array to multiply.
            scalar: The scalar to multiply by.

        Returns:
            The scalar product of the array.
        """
        # Check if its a vector
        if len(SNumPy.shape(array)) == 1:
            return [element * scalar for element in array]
        # Else, its a matrix
        else:
            return [
                [element * scalar 
                 for element in row] 
                for row in array
            ]

    @staticmethod
    def aug_matrix(matrix: list[list], vector: list) -> list[list]:
        """
        Append a vector as a new column to the right of a matrix.

        Args:
            matrix: The coefficient matrix to append to.
            vector: The constant vector to append.

        Returns:
            The augmented matrix.            
        """
        if len(matrix) != len(vector):
            raise ValueError(
                "Length of vector must match the number of rows in the matrix")

        return [row + [vector[i]] for i, row in enumerate(matrix)]

    @staticmethod
    def gaussian_elimination(matrix: list[list], vector: list, debug: bool = False) -> list[float]:
        """
        Solve a system of linear equations using Gaussian Elimination.

        This function applies Gaussian Elimination to solve the system of linear equations represented by the matrix equation Ax = B, where A is a square coefficient matrix, and B is a constant vector. 
        The function includes forward elimination with partial pivoting and backward substitution. 
        It rounds the solution vector to the nearest integer if the element is within a range 1e-15 (0.000000000000001) of the nearest integer.
        It checks for non-square matrices, singular matrices, and mismatched dimensions between the matrix and vector.

        Args:
            matrix : List[List[float]]
                Coefficient matrix (A), a 2D list representing the coefficients of the linear equations. Must be square (NxN).
            vector : List[float]
                Constant vector (B), a list representing the constant terms of the linear equations. Length must match the number of rows in 'matrix'.
            debug : Boolean (default is False)
                If True, prints the augmented matrix and upper triangular matrix.

        Returns:
            x : List[float]
                Solution vector (x) to the system Ax = B. Returns a list of float values representing the solution.
                The solution vector is rounded to the nearest integer if the element is within a very small range (0.0001) of the nearest integer.

        Raises:
            ValueError
                If the matrix is not square, if the matrix is singular (determinant is zero), or if the dimensions of the matrix and vector do not match.
        """
        # Validate the input
        Validator.validate_shape_for_operations(
            matrix, vector, 'gaussian_elimination')

        # Create necessary variables
        n = len(vector)
        m = n - 1
        x = SNumPy.zeros(n)  # Initialize solution vector with zeros

        # Augmented matrix
        augmented_matrix = SNumPy.aug_matrix(matrix, vector)

        if debug:
            print(f"Augmented Matrix: \n {augmented_matrix}")

        # Forward elimination
        for i in range(n):
            # Partial pivoting for numerical stability (dealing with floating point errors)
            max_row = max(range(i, n), key=lambda r: abs(
                augmented_matrix[r][i]))
            # Max row is moved to the top
            augmented_matrix[i], augmented_matrix[max_row] = augmented_matrix[max_row], augmented_matrix[i]

            # Check for singular matrix (pivot near zero)
            if abs(augmented_matrix[i][i]) < 1e-15:
                raise ValueError("Matrix is singular")

            # Eliminate the entries below the current pivot
            for j in range(i + 1, n):
                scaling_factor = augmented_matrix[j][i] / \
                    augmented_matrix[i][i]
                augmented_matrix[j] = SNumPy.subtract(
                    augmented_matrix[j], SNumPy.scalar_multiply(augmented_matrix[i], scaling_factor))

        if debug:
            print(f"Upper Triangular Matrix: \n {augmented_matrix}")

        # Backward substitution
        x[m] = augmented_matrix[m][n] / augmented_matrix[m][m]

        for i in range(m - 1, -1, -1):
            x[i] = augmented_matrix[i][n]
            for j in range(i + 1, n):
                x[i] = x[i] - augmented_matrix[i][j] * x[j]
            x[i] = x[i] / augmented_matrix[i][i]

        # Return the solution vector
        solution_vector = []
        for i in x:
            nearest_int = round(i)
            # Check if the element is within a very small range (1e-15) of the nearest integer
            if abs(i - nearest_int) < 1e-15:
                # If it's close enough, append the nearest integer to the solution list
                solution_vector.append(nearest_int)
            else:
                # Else append the float value
                solution_vector.append(i)

        return solution_vector

Gaussian Elimination adopted from the code found in this video, modified to work with list and not arrays as well as using our own SNumPy methods. (StudySession, 2023)

StudySession (Director). (2023, February 11). Gauss Elimination With Partial Pivoting In Python | Numerical Methods. https://www.youtube.com/watch?v=DiZ0zSzZj1g


Rounding was taken from the book "Introduction to Computation and Programming Using Python"

In [52]:
# Test cases for SNumPy class that compeares the output with numpy
import numpy as np

array1 = [[1, 2, 3], [4, 5, 6]]
array2 = [[1, 2, 3], [4, 5, 6]]
array3 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
matrix = [[1, 2], [3, 5]]
vector = [1, 2]
print(
    f"""
        SNumPy: \n
        Shape: {SNumPy.shape(array1)} \n
        Ones: {SNumPy.ones(3, 3)} \n
        Zeros: {SNumPy.zeros(3, 3)} \n
        Reshape: {SNumPy.reshape([1, 2, 3, 4, 5, 6], (2, 3))} \n
        Append: {SNumPy.append(array1, array2, axis=1)} \n
        Get: {SNumPy.get(array1, (1, 1))} \n
        Add: {SNumPy.add(array1, array2)} \n
        Subtract: {SNumPy.subtract(array1, array2)} \n
        Dot Product: {SNumPy.dotproduct(array1, array3)} \n
        Scalar Multiply: {SNumPy.scalar_multiply(array1, 2)} \n
        Gaussian Elimination: {SNumPy.gaussian_elimination(matrix, vector)} \n

        Numpy: \n
        Shape: {np.shape(array1)} \n
        Ones: {np.ones((3, 3))} \n
        Zeros: {np.zeros((3, 3))} \n
        Reshape: {np.reshape([1, 2, 3, 4, 5, 6], (2, 3))} \n
        Append: {np.append(array1, array2, axis=1)} \n
        Get: {np.array(array1)[1, 1]} \n
        Add: {np.add(array1, array2)} \n
        Subtract: {np.subtract(array1, array2)} \n
        Dot Product: {np.dot(array1, array3)} \n
        Scalar Multiply: {np.multiply(array1, 2)} \n
        Solve: {np.linalg.solve(matrix, vector)} \n
    """)


        SNumPy: 

        Shape: (2, 3) 

        Ones: [[1, 1, 1], [1, 1, 1], [1, 1, 1]] 

        Zeros: [[0, 0, 0], [0, 0, 0], [0, 0, 0]] 

        Reshape: [[1, 2, 3], [4, 5, 6]] 

        Append: [[1, 2, 3, 1, 2, 3], [4, 5, 6, 4, 5, 6]] 

        Get: 5 

        Add: [[2, 4, 6], [8, 10, 12]] 

        Subtract: [[0, 0, 0], [0, 0, 0]] 

        Dot Product: [[30, 36, 42], [66, 81, 96]] 

        Scalar Multiply: [[2, 4, 6], [8, 10, 12]] 

        Gaussian Elimination: [-1, 1] 


        Numpy: 

        Shape: (2, 3) 

        Ones: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] 

        Zeros: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 

        Reshape: [[1 2 3]
 [4 5 6]] 

        Append: [[1 2 3 1 2 3]
 [4 5 6 4 5 6]] 

        Get: 5 

        Add: [[ 2  4  6]
 [ 8 10 12]] 

        Subtract: [[0 0 0]
 [0 0 0]] 

        Dot Product: [[30 36 42]
 [66 81 96]] 

        Scalar Multiply: [[ 2  4  6]
 [ 8 10 12]] 

        Solve: [-1.  1.] 

    
